# RAG Agent Creation Tutorial

## Overview
This notebook demonstrates how to create a **Retrieval-Augmented Generation (RAG) agent** using the Hyland Agent Platform API. You'll learn to:

1. **Authenticate** with the Hyland API using OAuth 2.0 client credentials
2. **Configure** a RAG agent with specific parameters
3. **Create** the agent via REST API calls
4. **Understand** the response and next steps

## Prerequisites
- Valid Hyland client credentials (`CLIENT_ID` and `CLIENT_SECRET`)
- Basic understanding of REST APIs and JSON
- Familiarity with Python (helpful but not required)

## Instructions
1. **Run cells sequentially** from top to bottom
2. **Read the explanations** before each code section
3. **Enter your credentials** when prompted (they will be hidden)
4. **Check the output** after each cell execution

---

## Step 1: Authenticate and Obtain an JWT Access Token

This cell imports the necessary Python libraries and handles authentication with the Hyland API.

**When you run this cell:**
1. You'll be prompted for your CLIENT_ID and CLIENT_SECRET (input is hidden)
2. An access token is automatically retrieved
3. You're ready to create your RAG agent!

In [ ]:
import json
import urllib
import urllib.parse
import urllib.request
import requests
from getpass import getpass

client_id = getpass("Please Enter your CLIENT_ID: ")
client_secret = getpass("Please Enter your CLIENT_SECRET: ")

try:
    req = urllib.request.Request(
        "https://auth.iam.dev.experience.hyland.com/idp/connect/token", 
        data=urllib.parse.urlencode({
            "client_id": client_id,
            "grant_type": "client_credentials",
            "client_secret": client_secret,
            "scope": "hxp environment_authorization"
        }).encode("utf-8"), 
        headers={"Content-Type": "application/x-www-form-urlencoded"}, 
        method="POST")
    with urllib.request.urlopen(req) as response_from_auth:            
        response_data = json.loads(response_from_auth.read().decode("utf-8"))
        access_token = response_data.get("access_token")
        print("✅ Access token obtained successfully!")     
except Exception as e:
    print(f"❌ Failed to get token: {str(e)}")

Please enter your Hyland API credentials
Your input will be hidden for security


Enter your CLIENT_ID:  ········
Enter your CLIENT_SECRET:  ········


Credentials entered successfully!
Getting access token...
✅ Access token obtained successfully!


## Step 2: Configure Your RAG Agent

The next cell defines the configuration for your RAG agent. This JSON payload specifies how your agent will behave.

**Key Configuration Options:**
- **`name`**: A friendly name for your agent ("DocumentHelper")
- **`description`**: What your agent does ("Document assistant")
- **`agentType`**: Set to "rag" for Retrieval-Augmented Generation
- **`limit`**: Maximum number of documents to retrieve (10)
- **`llm_model_id`**: The AI model to use (Amazon Nova Micro)
- **`system_prompt`**: Instructions that guide how the AI responds
- **`temperature`**: Controls creativity (0.7 = balanced)
- **`max_tokens`**: Maximum response length (4000 tokens)

**💡 Tip:** You can modify these values to customize your agent's behavior before running the cell.

In [ ]:
payload = {
    "name": "DocumentHelper",
    "description": "Document assistant",
    "agentType": "rag",
    "notes": "Initial version - shows all configuration options",
    "config": {
        "filter_value": {},
        "limit": 10,
        "llm_model_id": "amazon.nova-micro-v1:0",
         "system_prompt": """
Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer:
""",
        "inference_config": {
            "temperature": 0.7,
            "max_tokens": 4000
        }
    }
}

## Step 3: Create Your RAG Agent

This is the main API call that creates your RAG agent on the Hyland platform.

**The response will include:**
- Your new agent's unique ID
- Confirmation of the configuration
- Any validation messages

In [ ]:
auth_headers ={
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': f'Bearer {access_token}'
}

response = requests.post(
    "https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents", 
    json=payload, 
    headers=auth_headers)

print(f"Status Code: {response.status_code}")
print(f"Response: {response.text}")

# Extract agent ID if creation was successful
agent_id = None
if response.status_code == 201:
    try:
        agent_id = response_data = response.json().get("id")
        if agent_id:
            print(f"\n✅ Agent created successfully! Agent ID: {agent_id}")
        else:
            print("\n⚠️ Agent created but no ID found in response")
    except Exception as e:
        print(f"\n❌ Error parsing response: {e}")
else:
    print(f"\n❌ Failed to create agent. Status: {response.status_code}")

Status Code: 201
Response: {"id":"14b2cd54-df91-4f76-9e77-9fa2b3e9d366","type":"rag","name":"DocumentHelper","description":"Document assistant","status":"CREATED","isGlobalAgent":false,"currentVersionId":"c2ac7c80-61ae-4b17-9479-25da69786d8c","createdAt":"2025-08-11T13:16:38.041045+00:00","createdBy":"f6822a26-880e-4358-96e3-a019a42121cd","modifiedAt":"2025-08-11T13:16:38.133123+00:00","modifiedBy":"f6822a26-880e-4358-96e3-a019a42121cd"}

✅ Agent created successfully!
🆔 Agent ID: 14b2cd54-df91-4f76-9e77-9fa2b3e9d366


## Step 4: Invoke Your RAG Agent with Conversation

Now that your agent is created, you can test it using a conversational format with message history! This simulates a real conversation where the agent can reference previous exchanges.

**Conversation Format:**
- **`messages`**: Array of conversation turns with roles and content
- **`role`**: Either "user" (human) or "assistant" (AI agent)
- **`content`**: The actual message text
- **`filterValue`**: Additional filtering criteria (can be empty)

**How it works:**
1. Agent processes the entire conversation history
2. Uses context from previous messages to inform responses  
3. Searches relevant documents based on the latest user question
4. Provides contextually aware answers

**💡 Example Conversation:**
- User asks about sick leave policy
- Agent responds with policy details
- User follows up asking for more HR documents
- Agent understands the context and provides related information

In [ ]:
# Check if we have a valid agent ID from the previous step
if not agent_id:
    print("❌ No agent ID available. Please run Step 3 successfully first.")
else:
    print(f"✅ Found agent ID: {agent_id}")
    
    # Conversation payload with message history (from invoke_rag_agent notebook)
    conversation_payload = {
        "messages": [
            {
                "role": "user",
                "content": "What is the company's sick leave policy?"
            },
            {
                "role": "assistant", 
                "content": "The company's sick leave policy allows employees to take a certain number of sick days per year. Please refer to the employee handbook for specific details and eligibility criteria."
            },
            {
                "role": "user",
                "content": "Tell me more about HR docs?"
            }
        ],
        "filterValue": {}
    }
    
    # Make the invoke request using the latest version endpoint
    invoke_url = f"https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents/{agent_id}/versions/latest/invoke"
    
    print(f"🤖 Invoking agent with conversation: {agent_id}")
    print(f"🔗 Endpoint: {invoke_url}")
    print(f"💬 Conversation has {len(conversation_payload['messages'])} messages")
    print("📡 Making request...")
    
    try:
        invoke_response = requests.post(invoke_url, json=conversation_payload, headers=auth_headers)
        print(f"📊 Response Status: {invoke_response.status_code}")
        
        if invoke_response.status_code != 200:
            print(f"⚠️ Response content: {invoke_response.text}")
            
    except Exception as e:
        print(f"❌ Error making request: {str(e)}")
        print("🔧 Check your network connection and try again")

🔍 Checking agent ID availability...
✅ Found agent ID: 14b2cd54-df91-4f76-9e77-9fa2b3e9d366
🤖 Invoking agent with conversation: 14b2cd54-df91-4f76-9e77-9fa2b3e9d366
🔗 Endpoint: https://api.agents.ai.dev.experience.hyland.com/agent-platform/v1/agents/14b2cd54-df91-4f76-9e77-9fa2b3e9d366/versions/latest/invoke
💬 Conversation has 3 messages
📡 Making request...
📊 Response Status: 401
⚠️ Response content: {"detail":"Token has expired"}


## Step 5: Process Conversation Response

This cell processes the agent's response to the conversational query and displays it in a readable format.

**What to expect:**
- **Contextual Response**: The agent considers the full conversation history
- **Follow-up Awareness**: Understands "Tell me more about HR docs?" relates to the previous sick leave question
- **Document References**: Shows which documents were used to answer
- **Conversation Continuity**: Response builds on previous exchanges

**Response Elements:**
- **Agent's Answer**: The AI-generated response to the latest user message
- **Source Documents**: References to HR documents and policies used
- **Conversation Context**: How the agent interpreted the message history
- **Processing Metadata**: Information about retrieval and generation

In [ ]:
# Process the agent's conversation response
if 'invoke_response' in locals():
    if invoke_response.status_code == 200:
        try:
            response_data = invoke_response.json()
            
            print("🎉 SUCCESS! Here's your agent's conversational response:\n")
            print("=" * 70)
            
            # Show the conversation context
            print("💬 CONVERSATION CONTEXT:")
            for i, msg in enumerate(conversation_payload['messages'], 1):
                role_icon = "👤" if msg['role'] == 'user' else "🤖"
                print(f"{i}. {role_icon} {msg['role'].upper()}: {msg['content']}")
            
            print("\n" + "-" * 50)
            
            # Display the main response
            if 'response' in response_data:
                print("🤖 AGENT'S RESPONSE TO LATEST QUESTION:")
                print(f"'{conversation_payload['messages'][-1]['content']}'")
                print()
                print(response_data['response'])
                print("\n" + "-" * 50)
            
            # Display source information if available
            if 'sources' in response_data and response_data['sources']:
                print("\n📚 DOCUMENTS REFERENCED:")
                for i, source in enumerate(response_data['sources'], 1):
                    print(f"{i}. {source.get('title', 'Unknown Document')}")
                    if 'snippet' in source:
                        print(f"   📄 Excerpt: {source['snippet'][:120]}...")
                    if 'score' in source:
                        print(f"   🎯 Relevance: {source['score']:.2f}")
                print("\n" + "-" * 50)
            
            # Display processing metadata
            if 'metadata' in response_data:
                metadata = response_data['metadata']
                print(f"\n📊 PROCESSING DETAILS:")
                print(f"   💭 Conversation turns processed: {len(conversation_payload['messages'])}")
                if 'documents_retrieved' in metadata:
                    print(f"   📋 Documents retrieved: {metadata['documents_retrieved']}")
                if 'processing_time' in metadata:
                    print(f"   ⏱️ Processing time: {metadata['processing_time']}ms")
            
            print("=" * 70)
            
        except Exception as e:
            print(f"❌ Error processing response: {e}")
            print(f"Raw response: {invoke_response.text}")
    
    else:
        print(f"❌ Agent conversation failed!")
        print(f"Status Code: {invoke_response.status_code}")
        print(f"Error: {invoke_response.text}")

else:
    print("ℹ️ No conversation response to process. Run Step 6 first to invoke your agent.")

🎉 SUCCESS! Here's your agent's conversational response:

💬 CONVERSATION CONTEXT:
1. 👤 USER: What is the company's sick leave policy?
2. 🤖 ASSISTANT: The company's sick leave policy allows employees to take a certain number of sick days per year. Please refer to the employee handbook for specific details and eligibility criteria.
3. 👤 USER: Tell me more about HR docs?

--------------------------------------------------
🤖 AGENT'S RESPONSE TO LATEST QUESTION:
'Tell me more about HR docs?'

{'choices': [{'message': {'role': 'assistant', 'content': "In addition to the Hyland Industries Employee Handbook 2023, Hyland Industries provides specific Summary Plan Descriptions (SPDs) for various benefit programs. These SPDs contain detailed information regarding the company's sick leave policy and other related policies.\n\nFor the sick leave policy, the relevant SPD would outline the specifics of how sick leave benefits are accrued, used, and calculated. It would detail the eligibility criteria, 

## Complete Workflow Summary

### 🎯 What You've Accomplished
You've successfully:
1. **Set up authentication** with the Hyland API and obtained access token
2. **Configured a RAG agent** with custom parameters
3. **Created the agent** on the platform
4. **Tested the agent** with conversational queries

### ✅ If Everything Worked
Your RAG agent is now ready for production use! You can:
- **Integrate it into applications** using the agent ID
- **Scale up document processing** by uploading more content
- **Monitor performance** through the Hyland dashboard
- **Modify configuration** by creating new agent versions

---

## Troubleshooting Guide

### 🔧 Agent Creation Issues (Step 3)

**Status 201: Success** ✅
- Agent created successfully
- Agent ID extracted and ready for use

**Status 400: Bad Request** ❌
- Check your payload configuration in Step 2
- Verify all required fields are present
- Ensure `llm_model_id` is valid

**Status 401: Unauthorized** ❌
- Verify your CLIENT_ID and CLIENT_SECRET in Step 1
- Check if credentials have expired
- Ensure you're using the correct environment

**Status 403: Forbidden** ❌
- Confirm your account has agent creation permissions
- Check if you're within usage limits

### 🤖 Agent Invocation Issues (Steps 4-5)

**Status 200: Success** ✅
- Query processed successfully
- Response contains agent's answer and sources

**Status 400: Bad Request** ❌
- Check your query format
- Ensure query is not empty
- Try a simpler question

**Status 404: Not Found** ❌
- Agent ID is incorrect or agent doesn't exist
- Verify Step 3 completed successfully
- Check if agent was deleted

**Status 401: Unauthorized** ❌
- Access token may have expired (tokens last ~1 hour)
- Re-run Step 1 to get a new token
- Check if headers are properly formatted

### 🔄 Testing Different Configurations

**To modify your agent:**
1. Go back to **Step 2** and change the `payload` dictionary
2. Re-run Step 3 to create a new agent
3. Test with Steps 4-5

**Common modifications:**
- Change `temperature` for more/less creative responses
- Adjust `limit` to retrieve more/fewer documents
- Modify `system_prompt` to change response style
- Try different `llm_model_id` options

### 🔄 Asking Multiple Questions

After completing all steps, you can:
1. **Rerun Steps 4-5** with different queries
2. **Change your question** in Step 4 and see different responses
3. **Compare responses** to understand your agent's capabilities

### 📚 Additional Resources
- [Hyland Agent Platform Documentation](https://docs.hyland.com)
- [RAG Agent Configuration Guide](https://docs.hyland.com/agents/rag)
- [API Reference](https://api.agents.ai.dev.experience.hyland.com/docs)
- [Best Practices for RAG Prompting](https://docs.hyland.com/agents/prompting)